In [308]:
# read csv file
from pandas import DataFrame, read_csv
import numpy as np
import tensorflow as tf
import pandas as pd
import operator

data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# merge train and test data, empty column = NaN
dummies_Type = pd.get_dummies(['test'], prefix= 'Type')
data_merge = data_train.append(data_test, ignore_index=True)

print('train',data_train.shape)
print('test',data_test.shape)
print('merge',data_merge.shape)

train (891, 12)
test (418, 11)
merge (1309, 12)


In [309]:
# create Title base on Name
import re
titleList = list()
for name in data_merge['Name']:
    nameList = re.split(', |\.', name)
    title = nameList[1]
    titleList.append(title)
titleCol = pd.DataFrame(data = titleList, columns=['Title'])
data_merge_addTitle = pd.concat([data_merge, titleCol], axis=1)

# add age range
rangeList = list()
for age in data_merge['Age']:
    if age < 10:
        rangeList.append('X')
    else:
        age = str(age)
        ageRange = age[:1] + 'X'
        rangeList.append(ageRange)
rangeCol = pd.DataFrame(data = rangeList, columns=['Range'])
data_merge_addRange = pd.concat([data_merge_addTitle, rangeCol], axis=1)

# adjust cabin
cabinList = list()
for cabin in data_merge['Cabin']:
    cabin = str(cabin)
    cabinNew = cabin[:1]
    cabinList.append(cabinNew)
cabinCol = pd.DataFrame(data = cabinList, columns=['CabinNew'])
data_merge_adjustCabin = pd.concat([data_merge_addRange, cabinCol], axis=1)


import math
fareList = list()
for fare in data_merge['Fare']:
    if math.isnan(fare):
        fareList.append(30)
    else:
        fareList.append(fare)
fareCol = pd.DataFrame(data = fareList, columns=['FareNew'])
data_merge_adjustFare = pd.concat([data_merge_adjustCabin, fareCol], axis=1)

In [310]:
# adjust to training format
data_final = data_merge_adjustFare
# # PLAN A
# 100 Train:  80.7142857143 Validate:  79.057591623
# dummy_list = ['Embarked','Sex','Parch','Pclass','Title','Range','CabinNew','SibSp']
# drop_list = ['FareNew','Age', 'Cabin', 'Fare', 'Name', 'Ticket']
# # B : REMOVE TITLE
# 100 Train:  82.1428571429 Validate:  79.057591623
# dummy_list = ['Embarked','Sex','Parch','Pclass','Range','CabinNew','SibSp']
# drop_list = ['FareNew','Age', 'Cabin', 'Fare', 'Name', 'Ticket','Title']
# # C: ADD FareNew
# 100 Train:  63.2857142857 Validate:  62.8272251309
# dummy_list = ['Embarked','Sex','Parch','Pclass','Title','Range','CabinNew','SibSp']
# drop_list = ['Age', 'Cabin', 'Fare', 'Name', 'Ticket']
# # D : Parch Pclass remain number
# 100 Train:  78.8571428571 Validate:  82.722513089
# dummy_list = ['Embarked','Sex','Title','Range','CabinNew','SibSp']
# drop_list = ['FareNew','Age', 'Cabin', 'Fare', 'Name', 'Ticket']
# # E : seperate all
# 100 Train:  78.8571428571 Validate:  79.057591623
dummy_list = ['Embarked','Sex','Parch','Pclass','Title','SibSp', 'Ticket', 'Cabin','Age', 'Fare']
drop_list = ['FareNew','Name','CabinNew','Range']
# # F : seperate all AND remove Fare
# 100 Train:  79.5714285714 Validate:  80.1047120419
# dummy_list = ['Embarked','Sex','Parch','Pclass','Title','SibSp', 'Ticket', 'Cabin','Age']
# drop_list = ['FareNew','Name','CabinNew','Range', 'Fare']
# # G
# 100 Train:  78.4285714286 Validate:  81.1518324607
# dummy_list = ['Embarked','Sex','Parch','Pclass','SibSp','Age','Cabin']
# drop_list = ['FareNew','Name', 'Fare','Title', 'Ticket', 'Range', 'CabinNew']


for attr in dummy_list:
    dummies = pd.get_dummies(data_final[attr], prefix= attr)
    data_final = pd.concat([data_final, dummies], axis=1)

data_final.drop(dummy_list, axis=1, inplace=True)
data_final.drop(drop_list, axis=1, inplace=True)
print('final data',data_final.shape)

# create csv data
data_merge_adjustFare.to_csv('DataAdjust.csv', index=False, header=True) 
data_final.to_csv('DataAdjustFinal.csv', index=False, header=True)

PassengerId = data_final.filter(regex='PassengerId').as_matrix()
Survived = data_final.filter(regex='Survived').as_matrix()
data_final.drop(['PassengerId','Survived'], axis=1, inplace=True)
dataMatrix = data_final.as_matrix()
print('seperate', PassengerId.shape, Survived.shape, dataMatrix.shape)

final data (1309, 1537)
seperate (1309, 1) (1309, 1) (1309, 1535)


In [311]:
# seperate to train and validate dataset
def one_hot(arr):
    minVal = min(arr)
    maxVal = max(arr)
    rangeVal = maxVal - minVal + 1
    result = []
    for val in arr:
        tempArr = [0] * int(rangeVal)
        tempArr[int(val - minVal)] = 1
        result.append(tempArr)
    return np.array(result)

trainData = dataMatrix[:700, :]
trainLabel = one_hot(Survived[:700])
validateData = dataMatrix[700:891, :]
validateLabel = one_hot(Survived[700:891])
testDataId = PassengerId[891:,0]
testData = dataMatrix[891:, :]
print('train', trainData.shape, trainLabel.shape)
print('validate', validateData.shape, validateLabel.shape)
print('test', testData.shape, testDataId.shape)

train (700, 1535) (700, 2)
validate (191, 1535) (191, 2)
test (418, 1535) (418,)


In [320]:
# training model : single layer
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

epochs = 10001
print_range = 500
decay_steps = 500
global_step = tf.placeholder(tf.int32)
learning_rate = tf.train.exponential_decay(learning_rate=0.9, global_step=global_step, decay_steps=decay_steps, decay_rate=0.9)
drop_rate = 0.5
atttribueNum = trainData.shape[1]
x = tf.placeholder(tf.float32, [None, atttribueNum])
W = tf.Variable(tf.zeros([atttribueNum, 2]))
b = tf.Variable(tf.zeros([2]))
# W = tf.Variable(tf.truncated_normal([atttribueNum, 2]))
# b = tf.Variable(tf.truncated_normal([2]))
y = tf.nn.softmax(tf.matmul(x, tf.nn.dropout(W, drop_rate)) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(epochs):
    sess.run(train_step, feed_dict = {x: trainData, y_: trainLabel, global_step:epochs})
#     for i in range(7):
#         subTrainData = trainData[100*i : 100*(i+1), :]
#         subTrainLabel = trainLabel[100*i : 100*(i+1), :]
#         sess.run(train_step, feed_dict = {x: subTrainData, y_: subTrainLabel, global_step:i})
    if epoch%print_range == 0:
        trainPrediction = sess.run(y, feed_dict={x: trainData})
        validatePrediction = sess.run(y, feed_dict={x: validateData})
        learningRate = sess.run(learning_rate, feed_dict={global_step:epochs})
        print(epoch, learningRate, 'Train: ', accuracy(trainPrediction, trainLabel), 'Validate: ', accuracy(validatePrediction, validateLabel))

0 0.478297 Train:  66.4285714286 Validate:  68.0628272251
500 0.478297 Train:  75.8571428571 Validate:  82.722513089
1000 0.478297 Train:  82.8571428571 Validate:  81.1518324607
1500 0.478297 Train:  85.7142857143 Validate:  83.2460732984
2000 0.478297 Train:  85.8571428571 Validate:  76.4397905759
2500 0.478297 Train:  87.1428571429 Validate:  81.1518324607
3000 0.478297 Train:  87.1428571429 Validate:  85.8638743455
3500 0.478297 Train:  87.4285714286 Validate:  84.8167539267
4000 0.478297 Train:  87.4285714286 Validate:  77.4869109948
4500 0.478297 Train:  92.0 Validate:  84.2931937173
5000 0.478297 Train:  92.7142857143 Validate:  76.4397905759
5500 0.478297 Train:  89.0 Validate:  81.1518324607
6000 0.478297 Train:  92.2857142857 Validate:  83.7696335079
6500 0.478297 Train:  92.1428571429 Validate:  84.8167539267
7000 0.478297 Train:  90.4285714286 Validate:  82.722513089
7500 0.478297 Train:  93.7142857143 Validate:  85.3403141361
8000 0.478297 Train:  91.7142857143 Validate:  8

In [317]:
# estimate for test data
def reverse_one_hot(matrix):
    result = list()
    for arr in matrix:
        max_index, max_value = max(enumerate(arr), key=operator.itemgetter(1))
        result.append(max_index)
    return result

drop_rate = 1
testResult = reverse_one_hot(sess.run(y, feed_dict={x: testData}))
dataSet = list(zip(testDataId,testResult))
df = pd.DataFrame(data = dataSet, columns=['PassengerId','Survived']) 
df.to_csv('test_result.csv', index=False, header=True)
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [209]:
# check training matrix
trainMatrix = sess.run(W, feed_dict = {x: trainData, y_: trainLabel})
col = list(data_final.columns)[2:]
coefData = list(zip(col,trainMatrix[:,1]))
coefCsv = pd.DataFrame(data = coefData, columns=['Attribute','Index']) 
coefCsv.to_csv('matrix.csv', index=False, header=True) 
coefCsv

,Attribute,Index
0,Embarked_S,0.188383
1,Sex_female,0.156794
2,Sex_male,-0.169340
3,Parch_0,0.604323
4,Parch_1,-0.710014
5,Parch_2,0.008369
6,Parch_3,0.108695
7,Parch_4,-0.014497
8,Parch_5,0.014171
9,Parch_6,-0.090511


In [322]:
# training model : 2 layer
global_step = tf.placeholder(tf.int32)
# learning_rate = tf.train.exponential_decay(learning_rate=0.9, global_step=global_step, decay_steps=50, decay_rate=0.9)
learning_rate = 0.1
drop_rate = 0.5
atttribueNum = trainData.shape[1]
hidden_layer_dimension = 1024

# layer 1
x = tf.placeholder(tf.float32, [None, atttribueNum])
W1 = tf.Variable(tf.truncated_normal([atttribueNum, hidden_layer_dimension]))
b1 = tf.Variable(tf.random_normal([hidden_layer_dimension]))
y1 = tf.nn.relu(tf.matmul(x, tf.nn.dropout(W1, drop_rate)) + b1)
# layer 2
W2 = tf.Variable(tf.truncated_normal([hidden_layer_dimension, 2]))
b2 = tf.Variable(tf.random_normal([2]))
y2 = tf.nn.softmax(tf.matmul(y1, tf.nn.dropout(W2, drop_rate)) + b2)

y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y2))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
# train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

epochs = 5001
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(epochs):
    sess.run(train_step, feed_dict = {x: trainData, y_: trainLabel, global_step:epochs})
#     for i in range(7):
#         subTrainData = trainData[100*i : 100*(i+1), :]
#         subTrainLabel = trainLabel[100*i : 100*(i+1), :]
#         sess.run(train_step, feed_dict = {x: subTrainData, y_: subTrainLabel, global_step:i})
    if epoch%100 == 0:
        trainPrediction = sess.run(y2, feed_dict={x: trainData})
        validatePrediction = sess.run(y2, feed_dict={x: validateData})
        print(epoch, 'Train: ', accuracy(trainPrediction, trainLabel), 'Validate: ', accuracy(validatePrediction, validateLabel))

0 Train:  44.1428571429 Validate:  58.1151832461
100 Train:  58.5714285714 Validate:  55.497382199
200 Train:  54.7142857143 Validate:  64.3979057592
300 Train:  62.5714285714 Validate:  65.9685863874
400 Train:  62.7142857143 Validate:  73.2984293194
500 Train:  55.2857142857 Validate:  63.8743455497
600 Train:  70.2857142857 Validate:  71.2041884817
700 Train:  75.1428571429 Validate:  73.2984293194
800 Train:  74.2857142857 Validate:  69.109947644
900 Train:  71.0 Validate:  68.5863874346
1000 Train:  66.0 Validate:  70.6806282723
1100 Train:  78.0 Validate:  67.0157068063
1200 Train:  73.5714285714 Validate:  64.9214659686
1300 Train:  63.1428571429 Validate:  79.057591623
1400 Train:  71.0 Validate:  71.7277486911
1500 Train:  71.8571428571 Validate:  75.3926701571
1600 Train:  70.7142857143 Validate:  75.3926701571
1700 Train:  72.7142857143 Validate:  77.4869109948
1800 Train:  76.5714285714 Validate:  65.9685863874
1900 Train:  76.1428571429 Validate:  68.0628272251
2000 Train: